In [5]:
#!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (4

In [20]:
from huggingface_hub import login

# подключение к hugging face hub для отправки своей модели
login()

In [2]:
from datasets import Dataset
import pandas as pd

# читаем обучающую выборку и превращаем ее в датасет
train = pd.read_csv("/content/train_spam.csv")

train['text_type'] = train['text_type'].map({'ham': 0, 'spam': 1})

train_ds = Dataset.from_pandas(train, split='train')

In [61]:
(train['text_type'] == 1).sum() / train.shape[0] * 100

29.54294139329156

> Порядка 30% данных - спамы \
> Особого дисбаланса классов не получается

In [64]:
print(train['text'].apply(lambda x: len(x)).max())
print(train['text'].apply(lambda x: len(x)).mean())

800
310.3835237744195


> Максимальная длина представленных текстов = 800 символов \
> Средняя длина представленных символов = 310 символов

In [4]:
from transformers import AutoTokenizer

# получаем токены базовоой BERT модели
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [5]:
def preprocess_data(data):
    '''
    Токенизируем наш текст с метками
    '''
    inputs = tokenizer(data['text'], padding='max_length', truncation=True, max_length=128)
    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': data['text_type']
    }

In [6]:
# преобразовываем наш текст
tokenized_train = train_ds.map(preprocess_data, batched=True).train_test_split(test_size=0.2)

Map:   0%|          | 0/16278 [00:00<?, ? examples/s]

In [7]:
from transformers import DataCollatorWithPadding

# используем данный класс, чтобы сравнять длину всех строк входных данных, чтобы модель не сломалась пр обучении
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
import evaluate

# инициализируем метрику точности
accuracy = evaluate.load("accuracy")

In [9]:
import numpy as np

# метрика для валидации модели при обучении
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [10]:
id2label = {0: 'ham', 1: 'spam'}
label2id = {'ham': 0, 'spam': 1}

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# берем предобученную модель БЕРТа, и дообучаем для нашей задачи классификации спама
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="vk_model_clf_distilbert",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train['train'],
    eval_dataset=tokenized_train['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.145100,0.130915,0.966216
2,0.073100,0.119021,0.968673
3,0.053100,0.133219,0.969595


TrainOutput(global_step=2442, training_loss=0.08443207080787953, metrics={'train_runtime': 510.9114, 'train_samples_per_second': 76.463, 'train_steps_per_second': 4.78, 'total_flos': 1293742848973824.0, 'train_loss': 0.08443207080787953, 'epoch': 3.0})

In [44]:
from tqdm import tqdm

# читаем тестовые данные и делаем на них предсказание
test = pd.read_csv('/content/test_spam.csv')
classifier = pipeline("sentiment-analysis", model="Paral1ax/vk_model_clf_distilbert")

predictions = np.array([])
for txt in tqdm(test['text']):
    predictions = np.append(predictions, classifier(txt))
# оставляем только метку предсказания
test['score'] = [pred['label'] for pred in predictions]
# скачиваем файл
test.to_csv("output.csv", index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 4070/4070 [11:55<00:00,  5.69it/s]
